In [157]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd, numpy as np, yfinance as yf
from datetime import datetime

In [ ]:
START_DATE   = "2010-01-01"
END_DATE     = datetime.today().strftime("%Y-%m-%d")
FORMATION    = 12          # look-back window (months)
SKIP         = 1           # skip window  (months)
HOLD         = 1           # holding period (months)
METRIC       = "raw"       # "risk" → mean/vol  |  "raw" → cumulative return
TOP_DECILE   = 0.10        # winners
BOT_DECILE   = 0.10        # losers
INIT_CAPITAL = 1_000_000   # rupees – used for trade P&L
FILENAME = f"momentum_backtest_results_nifty50_hold_{HOLD}_mo.xlsx"

In [ ]:
TICKERS = ['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']

In [ ]:
# DATA DOWNLOAD 
def fetch_prices(tickers, start, end):
    out = {}
    for tk in (t.strip() for t in tickers):
        if not tk:
            continue
        df = yf.download(tk, start=start, end=end,
                         auto_adjust=True, progress=False,
                         multi_level_index=False)
        if not df.empty:
            out[tk.replace(".NS", "")] = df["Close"]
    return pd.DataFrame(out)

daily_prices = fetch_prices(TICKERS, START_DATE, END_DATE)
if daily_prices.empty:
    raise RuntimeError("Price download failed – check tickers / internet")

monthly_prices  = daily_prices.resample("M").last()
monthly_returns = monthly_prices.pct_change()

In [ ]:
# STORAGE
equity_curve      = [1.0]
portfolio_records = []
trades            = []
per_stock_trades  = {}
open_positions    = []

all_months = monthly_returns.index
start_idx  = FORMATION + SKIP

In [ ]:
for idx in range(start_idx, len(all_months)):
    date = all_months[idx]

    # ---------- 1. BOOK P&L ON EXISTING POSITIONS ----------
    port_ret = 0.0
    to_close = []

    for pos in open_positions:
        s, w = pos['stock'], pos['weight']
        r = monthly_returns.get(s, pd.Series()).reindex(all_months).loc[date]
        if np.isnan(r):
            r = 0.0                          
        port_ret       += w * r
        pos['months_left'] -= 1             

        if pos['months_left'] == 0:         
            exit_px = monthly_prices.loc[date, s]
            if not np.isnan(exit_px):
                raw_ret   = (exit_px / pos['entry_price']) - 1
                trade_ret = raw_ret if w > 0 else -raw_ret
                profit_rs = INIT_CAPITAL * abs(w) * trade_ret

                trade = {
                    "Entry_Date" : pos['entry_date'].strftime("%Y-%m-%d"),
                    "Exit_Date"  : date.strftime("%Y-%m-%d"),
                    "Stock"      : s,
                    "Side"       : "Long" if w > 0 else "Short",
                    "Entry_Price": round(pos['entry_price'], 2),
                    "Exit_Price" : round(exit_px, 2),
                    "Return_% "  : round(trade_ret * 100, 2),
                    "Profit_RS"  : round(profit_rs, 2),
                    "Weight"     : round(w, 4),
                    "Mean_MonRet": round(pos['mean_ret'], 6),
                    "Volatility" : round(pos['vol_ret'], 6),
                    "Score"      : round(pos['score'], 6)
                }
                trades.append(trade)
                per_stock_trades.setdefault(s, []).append(trade)
            to_close.append(pos)

    for pos in to_close:                     
        open_positions.remove(pos)

    equity_curve.append(equity_curve[-1] * (1 + port_ret))

    formation_end   = date - pd.DateOffset(months=SKIP)
    formation_start = formation_end - pd.DateOffset(months=FORMATION-1)
    slice_ = monthly_returns.loc[formation_start:formation_end]

    mean_ret = slice_.mean()
    vol_ret  = slice_.std().replace(0, np.nan)   

    if METRIC == "risk":
        score = mean_ret / vol_ret
    elif METRIC == "raw":
        score = (1 + slice_).prod() - 1
    else:
        raise ValueError("METRIC must be 'risk' or 'raw'")

    score = score.dropna()
    n_stocks = len(score)

    if n_stocks == 0:                         
        portfolio_records.append(
            {"Month": date.strftime("%Y-%m"),
             "Year" : date.year,
             "Portfolio_Return": port_ret})
        continue                              

    n_long  = max(1, round(n_stocks * TOP_DECILE))
    n_short = max(1, round(n_stocks * BOT_DECILE))

    winners = score.nlargest(n_long).index.tolist()
    losers  = score.nsmallest(n_short).index.tolist()

    scale = 1 / HOLD
    new_weights = {s: +scale/len(winners) for s in winners}
    new_weights.update({s: -scale/len(losers) for s in losers})

    for s, w in new_weights.items():
        entry_px = monthly_prices.loc[date, s]
        if np.isnan(entry_px):
            continue
        open_positions.append({
            'stock'       : s,
            'weight'      : w,
            'months_left' : HOLD,
            'entry_price' : entry_px,
            'entry_date'  : date,
            'mean_ret'    : mean_ret.get(s, np.nan),
            'vol_ret'     : vol_ret.get(s, np.nan),
            'score'       : score[s]
        })

    current_weights = {}
    for pos in open_positions:
        current_weights[pos['stock']] = current_weights.get(pos['stock'], 0) + pos['weight']

    rec = {"Month": date.strftime("%Y-%m"),
           "Year" : date.year,
           "Portfolio_Return": port_ret}
    for tk in monthly_prices.columns:
        rec[tk] = current_weights.get(tk, 0)
    portfolio_records.append(rec)

In [ ]:
port_df   = pd.DataFrame(portfolio_records)
port_ret  = port_df["Portfolio_Return"]

ann_ret  = port_ret.mean() * 12
ann_vol  = port_ret.std(ddof=0) * np.sqrt(12)
sharpe   = ann_ret / ann_vol if ann_vol else np.nan
cum      = np.cumprod(1 + port_ret)
max_dd   = (cum / cum.cummax() - 1).min()
win_rate = (port_ret > 0).mean()

print("-"*60)
print(f"BACKTEST SUMMARY (Start → {port_df['Month'].iloc[-1]})")
print(f"Total Months            : {len(port_ret)}")
print(f"Total Trades (roundtrip): {len(trades)}")
print(f"Annualised Return       : {ann_ret*100:6.2f}%")
print(f"Annualised Volatility   : {ann_vol*100:6.2f}%")
print(f"Sharpe Ratio            : {sharpe:6.3f}")
print(f"Maximum Drawdown        : {max_dd*100:6.2f}%")
print(f"Winning Months          : {win_rate*100:6.1f}%")
print("-"*60)

with pd.ExcelWriter(FILENAME, engine="openpyxl") as xl:
    pd.DataFrame(trades).to_excel(xl, sheet_name="Trades", index=False)
    for stock, tlist in per_stock_trades.items():
        pd.DataFrame(tlist).to_excel(xl, sheet_name=stock[:31], index=False)
    port_df.to_excel(xl, sheet_name="Portfolio", index=False)

------------------------------------------------------------
BACKTEST SUMMARY (Start → 2025-06)
Total Months            : 173
Total Trades (roundtrip): 1655
Annualised Return       :   4.44%
Annualised Volatility   :  26.82%
Sharpe Ratio            :  0.166
Maximum Drawdown        : -62.08%
Winning Months          :   54.9%
------------------------------------------------------------


In [ ]:
n_ret = yf.download('^NSEI', start=START_DATE,
                    auto_adjust=True, progress=False,
                    multi_level_index=False)['Close'] \
            .resample('M').last().pct_change().dropna()
ann_ret_n, ann_vol_n = n_ret.mean()*12, n_ret.std()*np.sqrt(12)
print(f'NIFTY50  |  AnnRet {ann_ret_n:.2%}  Vol {ann_vol_n:.2%}  Sharpe {(ann_ret_n/ann_vol_n):.2f}')

print(f"\nExcel file created: {FILENAME}")

NIFTY50  |  AnnRet 12.07%  Vol 16.45%  Sharpe 0.73

Excel file created: momentum_backtest_results_nifty50_hold_1_mo.xlsx


In [ ]:
param_subset = {
    "FORMATION": FORMATION,
    "SKIP"     : SKIP,
    "HOLD"     : HOLD
}

summary_dict = {
    "Total_Months" : len(port_ret),
    "Total_Trades" : len(trades),
    "Ann_Return"   : round(ann_ret, 6),
    "Ann_Vol"      : round(ann_vol, 6),
    "Sharpe"       : round(sharpe, 3),
    "Max_Drawdown" : round(max_dd, 6),
    "Win_Rate"     : round(win_rate, 4)
}

FILENAME2 = f"parameters_and_summary1_nifty50_hold_{HOLD}_mo.xlsx"

pd.DataFrame([{**param_subset, **summary_dict}]).to_excel(FILENAME2,
    sheet_name="Overview",
    index=False,
    engine="openpyxl"
)
